# Test Correspondences

In [ ]:
%matplotlib widget
import numpy as np
import os
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import planeslam.io as io
from planeslam.general import NED_to_ENU, pc_plot_trace
from planeslam.scan import velo_pc_to_scan
from planeslam.registration import extract_corresponding_features, get_correspondences
from planeslam.point_cloud import velo_preprocess

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%load_ext autoreload
%autoreload 2

Read in airsim LiDAR and pose data

In [ ]:
# Read in point cloud data
pcpath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_20_2022', 'flightroom', 'run_8', 'pcs')
PCs = []
#len(os.listdir(pcpath))
#select_idxs = np.arange(0, 1000, 2)
select_idxs = np.arange(0, len(os.listdir(pcpath)))
for i in select_idxs:  
    filename = os.path.join(pcpath, 'pc_'+str(i)+'.npy')
    PC = np.load(filename)
    PCs.append(PC)

In [ ]:
# Read in pose data
posepath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_20_2022', 'flightroom', 'run_8', 'poses')
poses = []
for i in select_idxs:  
    filename = os.path.join(posepath, 'pose_'+str(i)+'.npy')
    pose = np.load(filename)
    poses.append(pose)

Extract planes 

In [ ]:
# Plot 2 scans
source_idx = 2
target_idx = 1
source = velo_pc_to_scan(velo_preprocess(PCs[source_idx], poses[source_idx]))
target = velo_pc_to_scan(velo_preprocess(PCs[target_idx], poses[target_idx]))

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

for t in source.plot_trace(show_normals=False):
    fig.add_trace(t, row=1, col=1)
#fig.add_trace(pc_plot_trace(PCs[source_idx]), row=1, col=1)

for t in target.plot_trace(show_normals=False):
    fig.add_trace(t, row=1, col=2)
#fig.add_trace(pc_plot_trace(PCs[target_idx]), row=1, col=2)

fig.update_layout(width=1600, height=500, scene=dict(aspectmode='data'), scene2=dict(aspectmode='data'))
fig.show()

correspondences = get_correspondences(source, target)
print(correspondences)

In [ ]:
source.remove_small_planes(area_thresh=0.1)
fig = go.Figure(data=source.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
n = len(source.planes) # source P
m = len(target.planes) # target Q
score_mat = np.zeros((n,m))
for i in range(n):
    for j in range(m):
        n1 = source.planes[i].normal
        n2 = target.planes[j].normal
        c1 = source.planes[i].center
        c2 = target.planes[j].center
        a1 = source.planes[i].area()
        a2 = target.planes[j].area()
        score_mat[i,j] = 100 * np.linalg.norm(n1 - n2) + np.linalg.norm(c1 - c2) + 0.1 * np.abs(a1 - a2)
        #score_mat[i,j] = np.linalg.norm(c1 - c2)

In [ ]:
from scipy.optimize import linear_sum_assignment

matches = linear_sum_assignment(score_mat)
matches

In [ ]:
from planeslam.registration import robust_GN_register

R_hat, t_hat = robust_GN_register(source, target)

print(R_hat)
print(t_hat)

In [ ]:
target.basis @ source.basis.T

In [ ]:
from planeslam.geometry.util import quat_to_R

R_1 = quat_to_R(poses[target_idx][3:])
R_2 = quat_to_R(poses[source_idx][3:])
R_gt = R_2 @ R_1.T

t_gt = poses[source_idx][:3] - poses[target_idx][:3]
print(R_gt)
print(t_gt)

In [ ]:
source.transform(R_hat, t_hat.flatten())

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

for t in source.plot_trace(show_normals=True):
    fig.add_trace(t, row=1, col=1)

for t in target.plot_trace(show_normals=True):
    fig.add_trace(t, row=1, col=2)

fig.update_layout(width=1200, height=500, scene=dict(aspectmode='data'), scene2=dict(aspectmode='data'))
fig.show()

In [ ]:
fig = go.Figure(data=source.plot_trace()+target.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()